In [1]:
# import numpy as np
# import random

# # Define TSP data (coordinates of cities)
# cities = np.array([
#     [0, 0],
#     [1, 3],
#     [2, 2],
#     [3, 1],
#     [5, 2],
#     [6, 0]
# ])
# # cities = load_dataset('dataset/large.csv')

# # Function to calculate the distance between two cities
# def distance(city1, city2):
#     return np.linalg.norm(city1 - city2)

# # Function to calculate total distance of a tour
# def tour_distance(tour):
#     total_distance = 0
#     for i in range(len(tour) - 1):
#         total_distance += distance(cities[tour[i]], cities[tour[i + 1]])
#     total_distance += distance(cities[tour[-1]], cities[tour[0]])  # Return to starting city
#     return total_distance

# # Initialize particles
# num_particles = 20
# num_iterations = 400
# particles = [random.sample(range(len(cities)), len(cities)) for _ in range(num_particles)]
# velocities = [random.sample(range(len(cities)), len(cities)) for _ in range(num_particles)]
# pBest = particles.copy()
# gBest = min(particles, key=tour_distance)

# # PSO parameters
# w = 0.5  # inertia weight
# c1 = 2.0  # cognitive weight
# c2 = 2.0  # social weight

# # PSO loop
# for iteration in range(num_iterations):
#     for i in range(num_particles):
#         # Update velocity
#         velocities[i] = w * np.array(velocities[i]) + \
#                         c1 * random.random() * (np.array(pBest[i]) - np.array(particles[i])) + \
#                         c2 * random.random() * (np.array(gBest) - np.array(particles[i]))
#         # Update position
#         particles[i] = sorted(range(len(velocities[i])), key=lambda k: velocities[i][k])
        
#         # Update personal best
#         if tour_distance(particles[i]) < tour_distance(pBest[i]):
#             pBest[i] = particles[i].copy()
            
#         # Update global best
#         if tour_distance(particles[i]) < tour_distance(gBest):
#             gBest = particles[i].copy()

# print("Best tour found:", gBest)
# print("Total distance:", tour_distance(gBest))


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
import math
import random

In [3]:
def read_cities(filepath):
    
    cities = np.loadtxt(filepath, delimiter=',')
    return cities

In [4]:
def score_solution(cities, solution):
    '''
    Calculate the total distance traveled by the given solution.
    This function scores a TSP solution by computing the total
    distance the salesperson would travel. Lower is better!
    The 'solution' array must contain indices into the 'cities'
    array. Also, the 'solution' array must visit each city exactly
    once!
    '''

    if len(solution) != len(cities):
        raise Exception(('Invalid solution: len(solution) is {}, ' + \
                'but it should be {}.').format(len(solution), len(cities)))

    if set(solution) != set(range(len(cities))):
        raise Exception('Invalid solution: The solution does not ' + \
                'visit each city exactly once!')

    dist = 0.0
    for i in range(len(solution)):
        p_prev = cities[solution[i-1]]
        p_here = cities[solution[i]]
        dist += euclidean(p_prev, p_here)
    return dist

In [5]:
def create_figure():
    '''
    Creates a figure which `visualize_solution()` will draw onto.
    '''
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    return fig, axes

In [6]:
def visualize_solution(cities, solution, fig=None, axes=None, block=True):
    '''
    Visualize the solution in a 2D plot.
    The 'cities' and 'solution' arguments are the same
    as to the `score_solution()` function.
    '''
    dist = score_solution(cities, solution) if len(solution) == len(cities) else float('NaN')

    if fig is None or axes is None:
        fig, axes = create_figure()
    ax1, ax2 = axes
    fig.suptitle('Total Distance: {}'.format(dist), fontsize=20)

    ax1.clear()
    ax1.scatter(cities[:,0], cities[:,1])

    if len(solution) == len(cities):
        path = np.hstack((solution, solution[0]))  # <-- the salesperson has to return home!
    else:
        path = solution
    ax2.clear()
    ax2.plot(cities[path,0], cities[path,1])
    ax2.scatter(cities[:,0], cities[:,1])

    if block:
        while plt.fignum_exists(fig.number):
            plt.pause(0.001)
    else:
        plt.pause(0.001)

In [7]:
def random_solution(cities):
    """
    Generate a random solution for the TSP problem.
    """
    return random.sample(range(len(cities)), len(cities))

In [8]:
def pso_tsp_solver(cities, start_index=0, swarm_size=20, max_iter=100, w=0.5, c1=1.5, c2=1.5):
    """
    Solve the TSP problem using Particle Swarm Optimization.
    """
    num_cities = len(cities)
    best_global_solution = None
    best_global_score = float('inf')

    # Initialize particles
    particles = []
    for _ in range(swarm_size):
        particle = random_solution(cities)
        # Ensure the start index is the specified city
        start_index_pos = particle.index(start_index)
        particle[0], particle[start_index_pos] = particle[start_index_pos], particle[0]
        particles.append({'position': particle, 'velocity': random_solution(cities)})

    for _ in range(max_iter):
        for particle in particles:
            score = score_solution(cities, particle['position'])
            if score < best_global_score:
                best_global_score = score
                best_global_solution = particle['position']

        for particle in particles:
            # Update velocity
            for i in range(num_cities):
                r1 = random.random()
                r2 = random.random()
                particle['velocity'][i] = (w * particle['velocity'][i] +
                                           c1 * r1 * (best_global_solution[i] - particle['position'][i]) +
                                           c2 * r2 * (random.choice(best_global_solution) - particle['position'][i]))

            # Update position
            particle['position'] = sorted(range(num_cities), key=lambda x: particle['position'][x] + particle['velocity'][x])

            # Ensure the start index is the specified city
            start_index_pos = particle['position'].index(start_index)
            particle['position'][0], particle['position'][start_index_pos] = particle['position'][start_index_pos], particle['position'][0]

    return best_global_solution

cities = read_cities('dataset/large.csv')

# Solve the TSP using PSO
solution = pso_tsp_solver(cities, start_index=5)

# Calculate the total distance of the solution
total_distance = score_solution(cities, solution)

print("Final solution:", solution)
print("Total distance traveled:", total_distance)


Final solution: [5, 953, 805, 836, 196, 713, 216, 103, 801, 460, 660, 835, 827, 306, 914, 132, 884, 28, 45, 373, 826, 506, 892, 318, 631, 372, 434, 589, 477, 887, 142, 862, 297, 30, 157, 635, 990, 771, 233, 922, 11, 577, 362, 528, 416, 677, 475, 984, 66, 537, 329, 489, 202, 650, 143, 188, 955, 259, 194, 309, 592, 147, 853, 556, 397, 282, 854, 474, 366, 987, 545, 320, 361, 350, 14, 476, 176, 278, 828, 761, 938, 76, 503, 178, 581, 667, 856, 394, 462, 175, 640, 122, 626, 59, 565, 128, 904, 714, 242, 865, 880, 32, 441, 261, 492, 703, 208, 95, 389, 920, 986, 67, 751, 304, 729, 249, 969, 590, 63, 369, 870, 711, 550, 661, 349, 286, 936, 816, 243, 236, 134, 579, 671, 799, 662, 603, 183, 384, 931, 921, 915, 482, 60, 722, 940, 17, 549, 810, 463, 681, 31, 852, 948, 473, 710, 523, 400, 796, 507, 834, 968, 73, 800, 407, 80, 408, 198, 783, 255, 331, 488, 620, 343, 803, 554, 131, 405, 942, 78, 472, 981, 930, 964, 260, 238, 849, 542, 732, 752, 289, 113, 493, 691, 431, 133, 468, 908, 57, 767, 404, 168,

In [9]:
486.2077578969698

486.2077578969698